In [2]:
import numpy as np
import pandas as pd

In [56]:
def sharpe_ratio(daily_returns):
    # get volatility
    vola_daily = np.std(daily_returns)
    # get returns
    total_returns = np.product(1 + daily_returns)
    returns_daily = np.exp(np.log(total_returns)/daily_returns.shape[0]) - 1
    return returns_daily / vola_daily
    
def sortino_ratio(daily_returns):
    # get volatility
    downsideReturns = daily_returns.copy()
    downsideReturns[downsideReturns > 0] = 0
    downsideVola = np.std(downsideReturns)
    # get returns
    total_returns = np.product(1 + daily_returns)
    returns_daily = np.exp(np.log(total_returns)/daily_returns.shape[0]) - 1
    return returns_daily / downsideVola

def slippage_costs(high, low, close, delta_weights, factor=0.05):
    """Taken as (High - Low) * factor, measured in absolute costs."""
    slippage = (high - low) / close * 0.05
    return abs(delta_weights * slippage)

def single_sim(data, old_weights, new_weights, portfolio_val, pred_prices, index):
    """Computes the estimated sharpe ratio for tomorrow given weights and predictions."""
    # index of today for convenience of accessing data's values
    TODAY, YTD = index, index-1
    
    # today's sharpe ratio (compute using past 5 days returns/std dev)
    # array of each stock's pct return today
    today_ret = (data.loc[TODAY, 'CLOSE'] - data.loc[YTD, 'CLOSE']) / data.loc[YTD, 'CLOSE']
    # array of weighted stock pct returns
    today_portfolio_ret = old_weights * today_ret
    # add today's new portfolio value to Series of portfolio values
    portfolio_val = portfolio_val.append(
        pd.Series(np.sum(today_portfolio_ret) * portfolio_val.iloc[-1])
    )
    # sharpe of pct returns over past 5 days (inc. today)
    today_sharpe = sharpe_ratio(portfolio_val.pct_change().tail(5))
    
    delta_weights = new_weights - old_weights
    # Use today's high-low to estimate slippage tomorrow
    slippage = slippage_costs(data.loc[TODAY, 'HIGH'], 
                              data.loc[TODAY, 'LOW'],
                              data.loc[YTD, 'CLOSE'],
                              delta_weights)
    
    # find tomorrow's returns based on new weights and predicted prices
    # ASSUMPTION: tomorrow's OPEN = today's CLOSE
    # array of each stock's predicted pct return tomorrow
    pred_ret = (pred_prices - data.loc[TODAY, 'CLOSE']) / data.loc[TODAY, 'CLOSE'] # % change
    # array of predicted weighted stock pct returns, deducting slippage for each stock
    pred_portfolio_ret = new_weights * pred_ret - slippage
    # sharpe of past 4 days (inc. today) and tomorrows predicted portfolio return
    pred_sharpe = sharpe_ratio(
        portfolio_val.append(
            pd.Series(np.sum(pred_portfolio_ret) * portfolio_val.iloc[-1]))\
                     .pct_change()\
                     .tail(5)
    )
    # maximize the difference here to greedily optimize
    return pred_sharpe - today_sharpe


Below cells are for testing purposes to make sure it runs correctly.

In [73]:
df = pd.read_pickle('./data_base_test.pkl').fillna(method='ffill').fillna(method='bfill')[['F_AD', 'F_AE', 'F_C']]

In [74]:
df.columns = df.columns.swaplevel()

In [97]:
old = np.random.normal(size=3)
new = np.random.normal(size=3)
portfolio_val = pd.Series([1e6] * df.shape[0])
pred_prices = df.tail(1)['CLOSE']

In [98]:
single_sim(df.reset_index(drop=True), old, new, portfolio_val, pred_prices, 30)

-0.026240826475426582

* Budget = SGD1000
* Stock A: SGD100 0.5 = 5 units,   predict SGD110 new_weight=0.6 = 6 units, est. profits=SGD60
* Stock B: SGD200 0.5 = 2.5 units, predict SGD250 new_weight=0.4 = 2 units, est. profits=SGD100
* True profit = (new_weight_A x Budget x price_change_A) + (new_weight_B x Budget x price_change_B)

Questions:
* How to set initial portfolio weights?

# Steps for greedy MC portfolio optimization

__Given predictions for next day + current portfolio weights, how to reweight the portfolio for Max. Sharpe__

(Naive)
1. Create random set of (delta) weights
2. Calculate total slippage cost (check formula in runts)
3. Using random weights, compute new portfolio estimated returns
4. Use past 5 days returns, find current sharpe ratio
5. Add next day's returns, find new sharpe ratio
6. Maximise difference in the ratios

# TODO: Create function to Monte Carlo simulate every day
(use single_sim for each day, then pick best set of weights)

In [ ]:
def mc_simulation(portfolio, random_state=12345, num_sims=1e6):




Sample code for MC portfolio optimization structure below:

In [2]:
# empty lists to store returns, variance and weights of imiginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
num_assets = portfolio.shape[1]
num_portfolios = 100000

#set random seed for reproduction's sake
np.random.seed(12345)

# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, daily_mean)
    volatility = np.sqrt(np.dot(weights, np.dot(daily_cov, weights)))
    sharpe = (returns - rf)/ volatility
    
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio
results = {'Returns': port_returns,
           'Volatility': port_volatility,
           'Sharpe Ratio': sharpe_ratio}

# extend original dictionary to accomodate each ticker and weight in the portfolio
for i, symbol in enumerate(names):
    results[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

array([ 0.46305446, -0.1298083 ,  1.04356154,  1.86002845,  1.77280517,
       -1.09453811,  0.19059528, -2.46233494, -0.08431552, -1.8870674 ,
       -0.38114357, -0.7243584 ,  0.48266087, -0.08356969, -0.06529756,
        0.11033144,  0.60356231, -0.40662759, -0.4754821 ,  0.62786559,
        0.5481597 ,  0.05239853,  0.97424779, -0.31027074, -1.41273487,
       -0.46230565, -0.08986267,  0.41671073,  0.79423572, -2.27910835,
        1.43016499,  0.8801555 , -0.36617876,  0.00471182,  0.24320903,
        0.57418533, -1.3532336 ,  1.36713463, -0.77480615,  0.6634038 ,
       -0.31926578, -0.85737327, -1.50334126,  0.26361554, -1.25975363,
        0.64825206,  0.76701135,  1.13038294, -0.19706875, -0.14134681,
        0.86965845, -0.47800553,  0.94566086, -0.34031788,  0.55310254,
        0.61349786, -1.51288333, -2.42951861,  0.57955658,  2.09797294,
        0.32083776,  0.78175145,  0.96816176, -0.82595979,  0.65010591,
       -0.21436279,  0.4198624 , -0.16650518,  0.1671119 ,  0.80